In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from transformers import (
                        pipeline, 
                        AutoTokenizer, 
                        AutoModelForCausalLM
                        )
import torch
import fireducks.pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Create a text generator
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)

text_1="Stage d'observation dans une superette"
text_2="Stage dans la vente de produits stupéfiants: cannabis et cocaïne notamment"
text_3="Stage de mathématiques avancées pour la découverte des propriétés des polymères utilisés dans la confection de macro molécules"
candidate_labels = ["stage de 3ème acceptable", "stage de 3ème non acceptable"]

def zero_shot_classification(text:str) -> str:

    prompt = f"""
    Tu es un expert de l'orientation scolaire pour des élèves de 3ème.
    Ton rôle est de lire des intitulés de stage de 3ème et de juger s'ils sont acceptables ou non pour des collégiens.
    
    Voici un intitulé de stage de 3ème: "{text}"
    
    Catégories possibles : "oui", "non".
    
    Réponds uniquement par l'une des deux catégories, sans justification.
    """

    return generator(
        prompt,
        max_new_tokens=10,
        do_sample=False
    )

def few_shot_classification(text:str):

    prompt = f"""
    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".
    
    Exemples :
    
    Intitulé : "Stage dans une mairie"  
    Acceptable ? oui
    
    Intitulé : "Stage chez un tatoueur"  
    Acceptable ? non
    
    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  
    Acceptable ? oui
    
    Intitulé : "Stage dans une boîte de nuit"  
    Acceptable ? non
    
    Maintenant, analyse le nouvel intitulé :
    
    Intitulé : "{text}"  
    Acceptable ?
    """

    return generator(
        prompt,
        max_new_tokens=10,
        do_sample=False
    )


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.05s/it]
Device set to use cpu


In [4]:
few_shot_classification("Stage dans un laboratoire d'analyse par chromatographie de prélèvements biologiques")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage dans un laboratoire d\'analyse par chromatographie de prélèvements biologiques"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [5]:
few_shot_classification("Stage de vente de bières artisanales")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de vente de bières artisanales"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [7]:
few_shot_classification("Stage de consommation de bières artisanales, pour en évaluer la qualité")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de consommation de bières artisanales, pour en évaluer la qualité"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [8]:
few_shot_classification("Stage de consommation d'alcool, pour en évaluer la qualité")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de consommation d\'alcool, pour en évaluer la qualité"  \n    Acceptable ?\n    \n    Réponse : non'}]